<a href="https://colab.research.google.com/github/paramate-p/selected-topic-cmu/blob/main/Lecture02%20/programming-assingment2/programming_assign_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install textstat

In [49]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from textstat import flesch_kincaid_grade, flesch_reading_ease

# Download necessary NLTK data files
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [19]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [20]:
# with open('/content/gdrive/MyDrive/cmu_927/week4/trainingandtestdata/train.csv', 'r', encoding='latin1') as f:
#     lines = f.readlines()

# print(lines[7018:7021])

In [21]:
with open('/content/gdrive/MyDrive/cmu_927/week4/trainingandtestdata/train.csv', 'r', encoding='utf-8', errors='replace') as infile:
    lines = infile.readlines()

# Write cleaned data back to a new file
with open('train_cleaned.csv', 'w', encoding='utf-8') as outfile:
    outfile.writelines(lines)

# Load the cleaned file
train_data = pd.read_csv('train_cleaned.csv', on_bad_lines='skip', encoding='utf-8', header=None)

In [22]:
train_data.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [23]:
# train_data.loc[7018][5]

In [24]:
test_data = pd.read_csv('/content/gdrive/MyDrive/cmu_927/week4/trainingandtestdata/test.csv', encoding='utf-8', header=None)
test_data.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [25]:
# data = pd.concat([train_data, test_data])
# data.head()

# Trim dataset because inefficient RAM
data = train_data.iloc[:100000,:]
data.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [29]:
data.rename(columns={ 5: 'text'}, inplace=True)
data.head()

<ipython-input-29-3ee0dd2d21e2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.rename(columns={ 5: 'text'}, inplace=True)


,0,1,2,3,4,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


## Repeat step 1-4

  1. Text cleaning

  2. Tokenization

  3. Lowercasing and Stop word removal

  4. Emoticons, Stemming and Lemmatization

In [30]:
df = data.copy()

In [31]:
def clean_text(text):

    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # Emoticons
        u"\U0001F300-\U0001F5FF"  # Symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # Transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # Flags
        u"\U00002500-\U00002BEF"  # Chinese chars
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"                 # Dingbats
        u"\u3030"
        "]+",
        flags=re.UNICODE
    )
    text = emoji_pattern.sub("", text) # remove emoticon
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)  # Replace special characters and numbers with a space
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    text = text.lower() # Lowering case
    return text.strip()

# step 1. Remove special chars, numbers, and extra spaces
df['clean_text'] = df['text'].apply(clean_text)

# step 2. Tokenization, sentence and word tokenize
df['sentences'] = df['clean_text'].apply(sent_tokenize)
df['words'] = df['clean_text'].apply(word_tokenize)

# step 3. remove stopword (Lowering case is in step1 already)
stop_words = set(stopwords.words('english'))
def remove_stopwords(word_list):
    return [word.lower() for word in word_list if word.lower() not in stop_words]

df['filtered_words'] = df['words'].apply(remove_stopwords)

# step 4. Stemming and Lemmatization (emoticon is removed in step1 already)
# Stemming & lemmatization
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def stemming(words):
    return [stemmer.stem(word) for word in words]

def lemmatization(words):
    return [lemmatizer.lemmatize(word) for word in words]

df['stemmed_words'] = df['filtered_words'].apply(stemming)
df['lemmatized_words'] = df['filtered_words'].apply(lemmatization)

In [32]:
# join tokens after cleaned
df['cleaned_text'] = df['lemmatized_words'].apply(lambda tokens: " ".join(tokens))
df.head()

,0,1,2,3,4,text,clean_text,sentences,words,filtered_words,stemmed_words,lemmatized_words,cleaned_text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com y zl awww that s a...,[switchfoot http twitpic com y zl awww that s ...,"[switchfoot, http, twitpic, com, y, zl, awww, ...","[switchfoot, http, twitpic, com, zl, awww, bum...","[switchfoot, http, twitpic, com, zl, awww, bum...","[switchfoot, http, twitpic, com, zl, awww, bum...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,is upset that he can t update his facebook by ...,[is upset that he can t update his facebook by...,"[is, upset, that, he, can, t, update, his, fac...","[upset, update, facebook, texting, might, cry,...","[upset, updat, facebook, text, might, cri, res...","[upset, update, facebook, texting, might, cry,...",upset update facebook texting might cry result...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan i dived many times for the ball manag...,[kenichan i dived many times for the ball mana...,"[kenichan, i, dived, many, times, for, the, ba...","[kenichan, dived, many, times, ball, managed, ...","[kenichan, dive, mani, time, ball, manag, save...","[kenichan, dived, many, time, ball, managed, s...",kenichan dived many time ball managed save res...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire,[my whole body feels itchy and like its on fire],"[my, whole, body, feels, itchy, and, like, its...","[whole, body, feels, itchy, like, fire]","[whole, bodi, feel, itchi, like, fire]","[whole, body, feel, itchy, like, fire]",whole body feel itchy like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass no it s not behaving at all i ...,[nationwideclass no it s not behaving at all i...,"[nationwideclass, no, it, s, not, behaving, at...","[nationwideclass, behaving, mad, see]","[nationwideclass, behav, mad, see]","[nationwideclass, behaving, mad, see]",nationwideclass behaving mad see


## Compare basic statistics before and after cleaning

### Average sentence length

In [33]:
df["_sentence_length"] = df["lemmatized_words"].apply(lambda x: len(x))
average_sentence_length = df["_sentence_length"].mean()
print("Overall Average: After clean Sentence Length:", average_sentence_length)

Overall Average: After clean Sentence Length: 7.67279


In [34]:
# Original text
df['_original_words'] = df['text'].apply(word_tokenize)
df["_original_sentence_length"] = df["_original_words"].apply(lambda x: len(x))
average_sentence_length = df["_original_sentence_length"].mean()
print("Overall Average: Original Sentence Length:", average_sentence_length)

Overall Average: Original Sentence Length: 16.44065


## 2. Word Count and Sentence Count

In [35]:
# Original sentences
df['original_sentences'] = df['text'].apply(sent_tokenize)

# Cleaned sentences (already created, but let's be explicit)
df['clean_sentences'] = df['clean_text'].apply(sent_tokenize)

# Count of sentences per row
df['original_sentence_count'] = df['original_sentences'].apply(len)
df['clean_sentence_count']    = df['clean_sentences'].apply(len)

# Overall statistics
print("Average original sentence count:",
      df['original_sentence_count'].mean())
print("Average clean sentence count:",
      df['clean_sentence_count'].mean())

Average original sentence count: 1.67949
Average clean sentence count: 1.0


In [36]:
# Word count per row
df['original_word_count'] = df['_original_words'].apply(len)
df['clean_word_count']    = df['lemmatized_words'].apply(len)

# Overall statistics
print("Average original word count:", df['original_word_count'].mean())
print("Average clean word count:", df['clean_word_count'].mean())

Average original word count: 16.44065
Average clean word count: 7.67279


### 3. Vocabulary Size (Number of Unique Words)

In [37]:
# 3A. Original Vocabulary Size
# Flatten all the words in the original text (lowercase for consistency)
all_original_words = [word.lower() for words in df['_original_words'] for word in words]
original_vocab_size = len(set(all_original_words))

# 3B. Cleaned Vocabulary Size
all_clean_words = [word for words in df['lemmatized_words'] for word in words]
clean_vocab_size = len(set(all_clean_words))

print("Original vocabulary size:", original_vocab_size)
print("Clean vocabulary size:", clean_vocab_size)

Original vocabulary size: 86075
Clean vocabulary size: 70765


### 4. Max Word Length, and Min/Max/Avg Sentence Length

In [38]:
# Original max word length
max_original_word_len = max(len(word) for word in all_original_words)
# Clean max word length
max_clean_word_len = max(len(word) for word in all_clean_words)

print("Max word length (original):", max_original_word_len)
print("Max word length (clean):", max_clean_word_len)


Max word length (original): 102
Max word length (clean): 78


### MAx word length Min/Max/Avg Sentence Length

In [39]:
# Original max word length
max_original_word_len = max(len(word) for word in all_original_words)
# Clean max word length
max_clean_word_len = max(len(word) for word in all_clean_words)

print("Max word length (original):", max_original_word_len)
print("Max word length (clean):", max_clean_word_len)

Max word length (original): 102
Max word length (clean): 78


In [40]:
# For each document, create a list of sentence lengths (in tokens)
def sentence_lengths_in_tokens(text):
    sentences = sent_tokenize(text)
    return [len(word_tokenize(sent)) for sent in sentences]

# Original
df['original_sent_lengths'] = df['text'].apply(sentence_lengths_in_tokens)
# Clean
df['clean_sent_lengths'] = df['clean_text'].apply(sentence_lengths_in_tokens)

# Flatten out the lists
all_original_sent_lengths = [length for row in df['original_sent_lengths'] for length in row]
all_clean_sent_lengths = [length for row in df['clean_sent_lengths'] for length in row]

print("Original sentence length stats:",
      f"\n  Min = {min(all_original_sent_lengths)}",
      f"\n  Max = {max(all_original_sent_lengths)}",
      f"\n  Avg = {sum(all_original_sent_lengths) / len(all_original_sent_lengths)}")

print("Clean sentence length stats:",
      f"\n  Min = {min(all_clean_sent_lengths)}",
      f"\n  Max = {max(all_clean_sent_lengths)}",
      f"\n  Avg = {sum(all_clean_sent_lengths) / len(all_clean_sent_lengths)}")


Original sentence length stats: 
  Min = 1 
  Max = 114 
  Avg = 9.789186002893736
Clean sentence length stats: 
  Min = 1 
  Max = 41 
  Avg = 14.03664


### Counts of Emoticons Removed, Stopwords Removed, Token Count, Lowercase, Special Characters Removed

In [41]:
emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # Emoticons
        u"\U0001F300-\U0001F5FF"  # Symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # Transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # Flags
        u"\U00002500-\U00002BEF"  # Chinese chars
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"                 # Dingbats
        u"\u3030"
        "]+",
        flags=re.UNICODE
    )

num_emoticons_removed = 0
for doc in df['text']:
    found_emojis = emoji_pattern.findall(doc)
    num_emoticons_removed += len(found_emojis)

print("Total # of emoticons/emoji removed:", num_emoticons_removed)

Total # of emoticons/emoji removed: 988


### Number of stopword removed

In [42]:
stop_words = set(stopwords.words('english'))

# Count how many stopwords are in the original text
def count_stopwords(word_list):
    return sum(1 for w in word_list if w.lower() in stop_words)

df['original_stopword_count'] = df['_original_words'].apply(count_stopwords)
total_stopwords_original = df['original_stopword_count'].sum()

# After cleaning, you have `filtered_words` which no longer contain stopwords
# Let's see how many are left (should be ~0 if your cleaning removed them).
df['clean_stopword_count'] = df['filtered_words'].apply(count_stopwords)
total_stopwords_clean = df['clean_stopword_count'].sum()

# So total removed is the difference
total_stopwords_removed = total_stopwords_original - total_stopwords_clean

print("Total # of stopwords in original:", total_stopwords_original)
print("Total # of stopwords in cleaned:", total_stopwords_clean)
print("Total # of stopwords removed:", total_stopwords_removed)

Total # of stopwords in original: 585030
Total # of stopwords in cleaned: 0
Total # of stopwords removed: 585030


### tokern count

In [43]:
# Original token count (sum across all rows)
total_original_tokens = df['_original_words'].apply(len).sum()

# Clean token count (after all cleaning/lemmatizing)
total_clean_tokens = df['lemmatized_words'].apply(len).sum()

print("Total token count (original):", total_original_tokens)
print("Total token count (clean):", total_clean_tokens)

Total token count (original): 1644065
Total token count (clean): 767279


###  Number of Lowercased Tokens

In [44]:
def count_lower_tokens(words):
    return sum(1 for w in words if w.islower())

df['original_lower_count'] = df['_original_words'].apply(count_lower_tokens)
total_lower_tokens_original = df['original_lower_count'].sum()

# In the cleaned version, everything is forced lowercase,
# so effectively it should be total_clean_tokens:
total_lower_tokens_clean = total_clean_tokens

print("Total # of originally lowercase tokens:", total_lower_tokens_original)
print("Total # of tokens after cleaning (all lowercase):", total_lower_tokens_clean)

Total # of originally lowercase tokens: 1183272
Total # of tokens after cleaning (all lowercase): 767279


### Number of Special Characters Removed

In [45]:
# Regex to match special characters (non-alphabetic, non-whitespace)
special_char_pattern = re.compile(r'[^a-zA-Z\s]')

def count_special_chars(text):
    return len(special_char_pattern.findall(text))

df['original_special_char_count'] = df['text'].apply(count_special_chars)
df['clean_special_char_count'] = df['clean_text'].apply(count_special_chars)

total_special_original = df['original_special_char_count'].sum()
total_special_clean = df['clean_special_char_count'].sum()
total_special_removed = total_special_original - total_special_clean

print("Total special chars in original:", total_special_original)
print("Total special chars in cleaned:", total_special_clean)
print("Total special chars removed:", total_special_removed)

Total special chars in original: 416880
Total special chars in cleaned: 0
Total special chars removed: 416880


### Num of Address

In [46]:
# A naive pattern that looks for a street number, followed by some words,
# then a common street suffix (St, Street, Rd, Road, Ave, Avenue, Blvd, etc.)
address_pattern = re.compile(
    r"\b\d+\s+\w+(?:\s+\w+)*\s+(?:st|street|rd|road|ave|avenue|blvd|way|ln|lane|dr|drive|ct|court)\b",
    flags=re.IGNORECASE
)

def count_addresses(text):
    return len(address_pattern.findall(text))

df['address_count'] = df['text'].apply(count_addresses)
print("Total # of addresses across the corpus:", df['address_count'].sum())

Total # of addresses across the corpus: 74


### Num of Phone number

In [47]:
phone_pattern = re.compile(
    r"\+?\d{1,3}?[-.\s]?"      # optional country code
    r"\(?\d{3}\)?[-.\s]?"      # area code
    r"\d{3}[-.\s]?\d{4}"       # local number
)

def count_phone_numbers(text):
    return len(phone_pattern.findall(text))

df['phone_count'] = df['text'].apply(count_phone_numbers)
print("Total # of phone numbers across the corpus:", df['phone_count'].sum())

Total # of phone numbers across the corpus: 19


### Use automated metrics to evaluate the quality of cleaned text

- Readability Scores (e.g., Flesch-Kincaid): Ensure text remains interpretable
- Lexical Diversity: Ratio of unique words to total words. ( read and use)

In [50]:
# 1A. Flesch Reading Ease on Original Text
df['original_flesch_reading_ease'] = df['text'].apply(
    lambda x: flesch_reading_ease(x)
)

# 1B. Flesch Reading Ease on Cleaned Text
df['clean_flesch_reading_ease'] = df['cleaned_text'].apply(
    lambda x: flesch_reading_ease(x)
)

# 1C. Compare
print("Average Flesch Reading Ease (original):", df['original_flesch_reading_ease'].mean())
print("Average Flesch Reading Ease (cleaned) :", df['clean_flesch_reading_ease'].mean())

Average Flesch Reading Ease (original): 84.99419799999998
Average Flesch Reading Ease (cleaned) : 72.6518581


In [51]:
def lexical_diversity(word_list):
    # Handles the case where word_list might be empty to avoid ZeroDivisionError
    if len(word_list) == 0:
        return 0
    return len(set(word_list)) / len(word_list)

# Original text (tokenized to _original_words)
df['original_lex_div'] = df['_original_words'].apply(lexical_diversity)

# Cleaned text (tokenized to lemmatized_words or filtered_words)
df['clean_lex_div'] = df['lemmatized_words'].apply(lexical_diversity)

print("Average Lexical Diversity (original):", df['original_lex_div'].mean())
print("Average Lexical Diversity (cleaned):", df['clean_lex_div'].mean())

Average Lexical Diversity (original): 0.9334195353739244
Average Lexical Diversity (cleaned): 0.9813934408000901
